In [1]:
# ======================  Colab setup & imports  =============================
!pip -q install --upgrade "openai>=1.0.0"
from google.colab import userdata
from openai import OpenAI
import textwrap, time

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
assert client.api_key, "Store the key via userdata.set_secret('OPENAI_API_KEY')."

# ======================  Settings  ==========================================
MODEL       = "gpt-4o-mini"
N_STEPS     = 5
TEMPERATURE = 0.0
TOP_P       = 0.0
SEED        = 42

BASE_TEXT = textwrap.dedent("""
    Alice was beginning to get very tired of sitting by her sister on the bank,
    and of having nothing to do...
""").strip()

PROMPT_TMPL = textwrap.dedent("""
You are a transformation engine.
TASK:
1. Paraphrase the text between triple back-ticks in fresh wording.
2. Append ONE new sentence that follows logically.
Return only the new paragraph.

```{text}```
""")

# ===== helper: extract chosen+top-2 for every output token ================
def extract_step_payload(resp):
    chosen   = []
    logps    = []
    top2_arr = []   # list of tuples: [(tok1, logp1), (tok2, logp2)]

    for tok_info in resp.choices[0].logprobs.content:
        chosen.append(tok_info.token)
        logps.append(tok_info.logprob)

        # top_logprobs is already sorted desc; take first two
        if tok_info.top_logprobs:
            top1 = (tok_info.top_logprobs[0].token,
                    tok_info.top_logprobs[0].logprob)
            top2 = (tok_info.top_logprobs[1].token,
                    tok_info.top_logprobs[1].logprob) if len(tok_info.top_logprobs) > 1 else None
        else:
            top1 = top2 = None
        top2_arr.append((top1, top2))
    return chosen, logps, top2_arr

# =================== run one deterministic chain ===========================
def run_chain(label, seed, steps):
    text, trace, fp, outputs = BASE_TEXT, [], None, []
    print(label + ':', end=' ')
    for _ in range(steps):
        r = client.chat.completions.create(
            model        = MODEL,
            messages     = [{"role":"user",
                          "content":PROMPT_TMPL.format(text=text)}],
            temperature  = TEMPERATURE,
            top_p        = TOP_P,
            seed         = seed,
            logprobs     = True,
            top_logprobs = 5
        )
        if fp is None:
            fp = r.system_fingerprint
        trace.append(extract_step_payload(r))
        text = r.choices[0].message.content.strip()
        outputs.append(text)          # <-- save each full paragraph
        print('.', end='', flush=True)

    print(f"  (fp {fp})")
    return trace, outputs             # return the chain of outputs

# ================== execute two runs =======================================
# -- run two chains
trace1, outs1 = run_chain("Run #1", SEED, N_STEPS)
time.sleep(1)
trace2, outs2 = run_chain("Run #2", SEED, N_STEPS)

# ================== locate first divergence & report =======================
div_step = div_tok = None
for step_idx, (step1, step2) in enumerate(zip(trace1, trace2), start=1):
    tokens1, _, _ = step1
    tokens2, _, _ = step2
    for tok_idx, (t1, t2) in enumerate(zip(tokens1, tokens2)):
        if t1 != t2:
            div_step, div_tok = step_idx, tok_idx
            break
    if div_step:
        break

if div_step is None:
    print(f"\nTwo runs are identical for all {N_STEPS} steps.")
else:
    # unpack data for the divergent token
    chosen1, logps1, top2s1 = trace1[div_step-1]
    chosen2, logps2, top2s2 = trace2[div_step-1]

    tok1, lp1 = chosen1[div_tok], logps1[div_tok]
    tok2, lp2 = chosen2[div_tok], logps2[div_tok]

    (alt1_1, alt1_lp), (alt1_2, alt1_lp2) = top2s1[div_tok]
    (alt2_1, alt2_lp), (alt2_2, alt2_lp2) = top2s2[div_tok]

    gap1 = alt1_lp - alt1_lp2 if alt1_lp2 is not None else float('nan')
    gap2 = alt2_lp - alt2_lp2 if alt2_lp2 is not None else float('nan')

    print(f"\nFirst divergence at step {div_step}, token #{div_tok}:")
    print("── Run #1 ──")
    print(f"  top-1: {repr(alt1_1):<15}  logp={alt1_lp:.6f}")
    print(f"  top-2: {repr(alt1_2):<15}  logp={alt1_lp2:.6f}")
    print(f"  gap  : {gap1:.6e} (top-1 − top-2)\n")

    print("── Run #2 ──")
    print(f"  top-1: {repr(alt2_1):<15}  logp={alt2_lp:.6f}")
    print(f"  top-2: {repr(alt2_2):<15}  logp={alt2_lp2:.6f}")
    print(f"  gap  : {gap2:.6e} (top-1 − top-2)\n")

    print("Δ(top-1 logp) between runs :", f"{abs(lp1 - lp2):.6e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 10.6 MB/s eta 0:00:00
Run #1: .....  (fp fp_34a54ae93c)
Run #2: .....  (fp fp_34a54ae93c)

First divergence at step 2, token #45:
── Run #1 ──
  top-1: ' lead'          logp=-1.056501
  top-2: ' spark'         logp=-1.056501
  gap  : 0.000000e+00 (top-1 − top-2)

── Run #2 ──
  top-1: ' spark'         logp=-0.979338
  top-2: ' lead'          logp=-1.104338
  gap  : 1.250000e-01 (top-1 − top-2)

Δ(top-1 logp) between runs : 7.716274e-02


In [4]:
from IPython.display import HTML, display
import html

# assumes outs1, outs2, div_step, N_STEPS already exist in session
def color_diff(text_a, text_b):
    """Return two HTML strings with differing tokens in red."""
    a_tokens, b_tokens = text_a.split(), text_b.split()
    out_a, out_b = [], []
    # compare up to max length
    max_len = max(len(a_tokens), len(b_tokens))
    for i in range(max_len):
        tok_a = a_tokens[i] if i < len(a_tokens) else ""
        tok_b = b_tokens[i] if i < len(b_tokens) else ""

        if tok_a == tok_b:
            out_a.append(html.escape(tok_a))
            out_b.append(html.escape(tok_b))
        else:
            if tok_a:
                out_a.append(f'<span style="color:red">{html.escape(tok_a)}</span>')
            if tok_b:
                out_b.append(f'<span style="color:red">{html.escape(tok_b)}</span>')
    return " ".join(out_a), " ".join(out_b)

# divergence paragraph
idx = div_step - 1 if div_step is not None else 0
div1_html, div2_html = color_diff(outs1[idx], outs2[idx])

# final paragraphs
final1_html, final2_html = color_diff(outs1[-1], outs2[-1])

html_block = f"""
<h3>Paragraphs at Divergence (Step {div_step})</h3>
<strong>Run #1</strong><br>
<p>{div1_html}</p>
<strong>Run #2</strong><br>
<p>{div2_html}</p>

<h3>Final Paragraphs after {N_STEPS} Iterations</h3>
<strong>Run #1</strong><br>
<p>{final1_html}</p>
<strong>Run #2</strong><br>
<p>{final2_html}</p>
"""

display(HTML(html_block))
